$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [1]:
from grasp_betas import read_results

betas = read_results()

In [4]:
means = betas.groupby("beta").mean()
means

,OF,time,improvs
beta,,,
-1.00,855.814815,33.751611,2.407407
0.00,856.611111,26.891191,1.962963
0.25,856.629630,28.787164,2.296296
0.50,854.351852,31.004062,2.074074
0.75,855.518519,36.939448,2.444444
1.00,855.574074,48.986284,2.796296


In [7]:
100 * abs(854.351852 - 856.611111) / 856.611111

0.263743835561811

In [9]:
100 * abs(31.004062 - 26.891191) / 26.891191

15.294491791010676